In [1]:
import dropbox
import datetime
import schedule
import numpy
import numpy
import string
import requests 
import sqlite3
import pandas as pd

from threading import Timer
import time
import sys

import csv

import math
import random
import matplotlib
# import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import animation
import os

import matplotlib.dates as mdate
from time import gmtime, strftime

from matplotlib.ticker import MultipleLocator
from matplotlib.ticker import FormatStrFormatter

import pytz
import warnings
from gekko import GEKKO

class HR_Getting:

    def E4(x, y, filetime):
        # Dropbox access token    
        dbx = dropbox.Dropbox('Dropbox key here')

        Str_filetime = filetime[-1].strftime('%Y-%m-%d-%H-%M')

        # Saving path
        download_path = '/Users/tomstanton 1/Documents/College/Stage 5/ME Project/Data/E4data/BPM/'+Str_filetime+'.txt'
        # File location in dropbox    
        path = '/IBI/IBIData'+Str_filetime+'.txt'

        # try download
        try:
            dbx.files_download_to_file(download_path,path,None)   
        except:
                print('IBI Data at '+Str_filetime+' does not exsit')
                pass
        else:
            # A file downloaded, reformat the file
            resval = numpy.loadtxt(download_path)

            with open(download_path, 'r') as f:
                sum = 0
                count = 0
                lines = f.readlines()  # read the content
                for line in lines:
                    for value in line.split():
                        value = value.strip(string.whitespace)
                        sum += float(value)
                        count += 1
                
                f.close()
                
            aveIBI = sum / count
            bpm = int(60 / aveIBI)
            d5_bpm = int(bpm/5)
 
            x.append(Str_filetime)
            y.append(bpm)
                
            with open('/Users/tomstanton 1/Documents/College/Stage 5/ME Project/Data/E4data/bpm_data.csv', 'a', newline='\n') as f:
                headers = ['time', 'bpm']
                csv_write = csv.writer(f)
                
                
                if not os.stat("/Users/tomstanton 1/Documents/College/Stage 5/ME Project/Data/E4data/bpm_data.csv").st_size:    
                    print('BPM Data file created')
                    csv_write.writerow(headers) # file doesn't exist yet, write a header
        
                csv_write.writerow([Str_filetime, bpm])
                f.close()
                
            print('Time: ', Str_filetime ,'. Avg HR: ', bpm,)



class EBikeData_Getting:
        
    def EBike(x_t,y_hP,filetime):
        # Dropbox access token    
        dbx = dropbox.Dropbox('Dropbox key here')

        # Saving path
        save_path = '/Users/tomstanton 1/Documents/College/Stage 5/ME Project/Data/Bikedata/Data.db'
        # File location in dropbox    
        path = '/EBike.db'

        # Re-formatting: datetime->String
        timeframe = filetime[-1].strftime('%Y-%m-%d %H:%M')
        csv_filetime = filetime[-1].strftime('%Y-%m-%d-%H-%M')

        # try download
        try:
            dbx.files_download_to_file(save_path,path,None)   
        except:
                pass
        else:
            # Save voltage of the motor and data which averaged
            conn = sqlite3.connect(save_path)
            cursor = conn.cursor()

            try:
                res_findData=cursor.execute("SELECT * FROM bikeData WHERE time LIKE '%%%s%%'" %timeframe) # find "key words" in column time of table bikeData
            except:
                conn.close()
                time.sleep(10)
                dbx.files_download_to_file(save_path,path,None)
                print('redownload')
                conn = sqlite3.connect(save_path)
                cursor = conn.cursor()

            try:
                if cursor.fetchone() is None:   # No corresponding data to the time, need Optimize
                    count = 0
                    print('Bike Data at '+csv_filetime+' does not exist')
                    pass
                else:
                    rows = cursor.fetchall()

                    sum_hP = 0
                    sum_ThrOut = 0
                    count = 0

                    for row in rows:
                        sum_hP += row[9]
                        sum_ThrOut += row[12]
                        count += 1

                conn.close()

                if count == 0:
                    pass

                else:
                    ave_hP = sum_hP/count
                    d5_hP = int(ave_hP/5)
                    ave_ThrOut = sum_ThrOut/count
                    d5_ThrOut = int(ave_ThrOut/5)
                    # export the data processed into file "export_EBikedata.csv"

                    x_t.append(csv_filetime)
                    y_hP.append(ave_hP)


                    with open("/Users/tomstanton 1/Documents/College/Stage 5/ME Project/Data/BikeData/EBikedata.csv", "a", newline='\n') as f:
                        headers = ['time', 'average humanPower','average ThrottleOutput']
                        writer = csv.writer(f)

                        if not os.stat("/Users/tomstanton 1/Documents/College/Stage 5/ME Project/Data/BikeData/EBikedata.csv").st_size:         
                            print('Bike Data file created')
                            writer.writerow(headers) # file doesn't exist yet, write a header

                        writer.writerow([csv_filetime,ave_hP,ave_ThrOut])
                        f.close()
                        print('Time: ', csv_filetime ,'. Avg HP: ', ave_hP)
            except:
                    count = 0
                    print('Bike Data at '+csv_filetime+' does not exist')


class Merging_Data:
    
    def MergeTable(x_t1,x_t3,y_hr,y_hP,u,t):
        
        # concentrate Ebike data and E4 data in one csv file 
        df3 = pd.read_csv("/Users/tomstanton 1/Documents/College/Stage 5/ME Project/Data/BikeData/EBikedata.csv")
        df1 = pd.read_csv("/Users/tomstanton 1/Documents/College/Stage 5/ME Project/Data/E4data/bpm_data.csv")
        merged = df1.merge(df3, on="time", how="outer").fillna("")
        merged = merged.sort_values(by='time')
        merged.to_csv("/Users/tomstanton 1/Documents/College/Stage 5/ME Project/Data/AllData.csv", index=False)
        
        dfHR = pd.DataFrame()
        dfHR['time']  = x_t1
        dfHR['bpm']  = y_hr
        dfHP = pd.DataFrame()
        dfHP['time']  = x_t3
        dfHP['hp']  = y_hP
        dfU = pd.DataFrame()
        dfU['time'] = t
        dfU['al'] = u 
        
        dfHR.drop_duplicates(subset ="time",keep = 'first', inplace = True) 
        dfHP.drop_duplicates(subset ="time",keep = 'first', inplace = True) 
        dfU.drop_duplicates(subset ="time",keep = 'first', inplace = True) 
        x_t1 = dfHR.time
        y_hr = dfHR.bpm
        x_t3 = dfHP.time
        y_hP = dfHP.hp
        t = dfU.time
        u = dfU.al
        
        currCycle = dfHR.merge(dfHP, on="time", how="outer").fillna("")
        currCycle = currCycle.merge(dfU, on="time", how="outer").fillna("")
        currCycle = currCycle.sort_values(by='time')
        currCycle.to_csv("/Users/tomstanton 1/Documents/College/Stage 5/ME Project/Data/CurrentCycleData.csv", index=False)

        
        

class SysID_MPC:
    
    def ModelAndControl(u,t,ukOld,filetime,y_hr,y_hp,x_t1,x_t3):
        

        warnings.filterwarnings("ignore")
        Str_filetime = filetime[-1].strftime('%Y-%m-%d-%H-%M')

        # Current Cycle Data
        currCycle = pd.read_csv("/Users/tomstanton 1/Documents/College/Stage 5/ME Project/Data/CurrentCycleData.csv")
        filename = currCycle.time[len(currCycle.time)-1]
        currCycleReadings = len(currCycle.time)
        currCycle2=currCycle.dropna(subset = ['bpm','hp'])
        currCycleReadings2 = len(currCycle.time)
        

        # Initialize X component values at time k
        x1k = currCycle.bpm[currCycleReadings-1]
        x3k = currCycle.hp[currCycleReadings-1]

        mpcFlag=0
        # Get previous prediction table if one exists
        if currCycleReadings>1:
            try:
                str_filetime = filetime[-2].strftime('%Y-%m-%d-%H-%M')
                filepath = '/Users/tomstanton 1/Documents/College/Stage 5/ME Project/Data/MPC_Predictions/'+str_filetime+'.csv'
                dfPredict = pd.read_csv(filepath)
            except:
                mpcFlag=1
            else:
                mpcFlag=0


        # Fill in missing data in current frame using predicted mpc value in last frame or initial set values    
        flag=0

        if numpy.isnan(x1k):
            flag=1
            if currCycleReadings<=1:
                x1k = 70+0.1
                y_hr.append(x1k)
                x_t1.append(Str_filetime)
            elif mpcFlag==0:
                x1k = round(dfPredict.bpm[1])+0.1
                y_hr.append(x1k)
                x_t1.append(Str_filetime)
            else:
                x1k = y_hr[-1]+0.1
                y_hr.append(x1k)
                x_t1.append(Str_filetime)
            print("Prediction/Previous HR selected =  ",x1k)

        if numpy.isnan(x3k):
            flag=1
            if currCycleReadings<=1:
                x3k = 0+0.1
                y_hP.append(x3k)
                x_t3.append(Str_filetime)
            elif mpcFlag==0:
                x3k = round(dfPredict.hp[1])+0.1
                y_hP.append(x3k)
                x_t3.append(Str_filetime)
            else:
                x3k = y_hP[-1]+0.1
                y_hP.append(x3k)
                x_t3.append(Str_filetime)
            print("Prediction/Previous HP selected = ",x3k)

        if flag==1:
            ccEdit= pd.read_csv("/Users/tomstanton 1/Documents/College/Stage 5/ME Project/Data/CurrentCycleData.csv")
            ccEdit.at[len(ccEdit.time)-1,'bpm'] = x1k
            ccEdit.at[len(ccEdit.time)-1,'hp'] = x3k
            ccEdit.to_csv("/Users/tomstanton 1/Documents/College/Stage 5/ME Project/Data/CurrentCycleData.csv", index=False)

        trainData = pd.read_csv("/Users/tomstanton 1/Documents/College/Stage 5/ME Project/Data/TrainingData3.csv")
        trainReadings = len(trainData.time)
        x1t = trainData.bpm
        x3t = trainData.hp
        ut = trainData.al

        x1h = y_hr
        x3h = y_hP
        uh=u
        
        
        
        # Parameter Identification
        # Initialize Model
        m = GEKKO()

        # Initialize Variables
        a13,a31,b1,b3  = [m.Var() for i in range(4)]
        e = m.Array(m.Var,((len(x1h))*2)) # error [recorded]
        et = m.Array(m.Var,((trainReadings)*2)) # error [training]


        # Equations
        # Recorded Data Constraints
        for i in range(0,trainReadings-1):
            # Training Data Constraints
            m.Equation(x1t[i+1]-x1t[i] == a13*x3t[i] + b1*ut[i]+et[i*2+1])
            m.Equation(x3t[i+1]-x3t[i] == a31*x1t[i] + b3*ut[i]+et[i*2+2])
        
        for i in range(0,len(x1h)-1):
            m.Equation(x1h[i+1]-x1h[i] == a13*x3h[i] + b1*uh[i]+e[i*2+1])
            m.Equation(x3h[i+1]-x3h[i] == a31*x1h[i] + b3*uh[i]+e[i*2+2])


        # Objective
        m.Obj(sum(numpy.square(et))+sum(numpy.square(e)))
        m.options.IMODE = 3 # MPU
        m.options.MAX_ITER = 10000
        m.solve(disp=False)

        a11 = 1
        a13 =  float(str(a13.value)[1:-1])
        a31 =  float(str(a31.value)[1:-1])
        a33 = 1
        b1 =  float(str(b1.value)[1:-1])
        b3 =  float(str(b3.value)[1:-1])
        print("a13=",a13,", a31=",a31,", b1=",b1,", b3=",b3)
        parameters = [[a11,a13,a31,a33,b1,b3]]
        parametersDF = pd.DataFrame(parameters, columns = ['a11','a13','a31','a33','b1','b3']) 
        filename = filetime[-1].strftime('%Y-%m-%d-%H-%M') 
        parametersDF.to_csv("/Users/tomstanton 1/Documents/College/Stage 5/ME Project/Data/Parameters/"+filename+'.csv', index=False)


        # MPC
        # Initialize Model
        n = GEKKO()

        # Constants
        bpmMax = 120

        # Initialize Variables
        x1k1 = n.Var(lb = 0, ub = 200)
        x3k1 = n.Var(lb = 0, ub = 1000) 
        uk = n.Var(lb = 0, ub = 1) 
        x1k2 = n.Var(lb = 0, ub = 200)
        x3k2 = n.Var(lb = 0, ub = 1000) 
        uk1 = n.Var(lb = 0, ub = 1) 
        x1k3 = n.Var(lb = 0, ub = 200)
        x3k3 = n.Var(lb = 0, ub = 1000) 
        uk2 = n.Var(lb = 0, ub = 1) 
#             x1k4 = n.Var(lb = 0, ub = 200)
#             x3k4 = n.Var(lb = 0, ub = 1000) 
#             uk3 = n.Var(lb = 0, ub = 1) 
#             x1k5 = n.Var(lb = 0, ub = 200)
#             x3k5 = n.Var(lb = 0, ub = 1000) 
#             uk4 = n.Var(lb = 0, ub = 1) 

        # Equations
        n.Equation(x1k1 == a11*x1k + a13*x3k + b1*uk)
        n.Equation(x3k1 == a31*x1k + a33*x3k + b3*uk)
        n.Equation(x1k1 <= bpmMax)
        n.Equation(x1k1 >= x1k)
        n.Equation(x1k2 == a11*x1k1 + a13*x3k1 + b1*uk1)
        n.Equation(x3k2 == a31*x1k1 + a33*x3k1 + b3*uk1)
        n.Equation(x1k2 <= bpmMax)
        n.Equation(x1k2 >= x1k1)
        n.Equation(uk1 <= uk)
        n.Equation(x1k3 == a11*x1k2 + a13*x3k2 + b1*uk2)
        n.Equation(x3k3 == a31*x1k2 + a33*x3k2 + b3*uk2)
        n.Equation(x1k3 <= bpmMax)
        n.Equation(x1k3 >= x1k2)
        n.Equation(uk2 <= uk1)
#             n.Equation(x1k4 == a11*x1k3 + a13*x3k3 + b1*uk3)
#             n.Equation(x3k4 == a31*x1k3 + a33*x3k3 + b3*uk3)
#             n.Equation(x1k4 <= bpmMax)
#             n.Equation(x1k4 >= x1k3)
#             n.Equation(uk3 <= uk2)
#             n.Equation(x1k5 == a11*x1k4 + a13*x3k4 + b1*uk4)
#             n.Equation(x3k5 == a31*x1k4 + a33*x3k4 + b3*uk4)
#             n.Equation(x1k5 <= bpmMax)
#             n.Equation(x1k5 >= x1k4)
#             n.Equation(uk4 <= uk3)

        # Objective
#             n.Obj(uk+uk1+uk2+uk3+uk4)
        n.Obj(uk+uk1+uk2)
        n.options.IMODE = 3
        n.options.MAX_ITER = 10000
        try:
            n.solve(disp=False)
        except:
            print("Couldn't find optimal assistance levels")
            uk = ukOld[len(ukOld)-1]
        else:
            x1k1 = float(str(x1k1.value)[1:-1])
            x1k2 = float(str(x1k2.value)[1:-1])
            x1k3 = float(str(x1k3.value)[1:-1])
#                 x1k4 = float(str(x1k4.value)[1:-1])
#                 x1k5 = float(str(x1k5.value)[1:-1])
            x3k1 = float(str(x3k1.value)[1:-1])
            x3k2 = float(str(x3k2.value)[1:-1])
            x3k3 = float(str(x3k3.value)[1:-1])
#                 x3k4 = float(str(x3k4.value)[1:-1])
#                 x3k5 = float(str(x3k5.value)[1:-1])
            uk = round(float(str(uk.value)[1:-1]),2)
            uk1 = round(float(str(uk1.value)[1:-1]),2)
            uk2 = round(float(str(uk2.value)[1:-1]),2)
#                 uk3 = round(float(str(uk3.value)[1:-1]),2)
#                 uk4 = round(float(str(uk4.value)[1:-1]),2)
            ukOld.append(uk)

            # Output
            prediction = [[0,x1k,x3k,uk],[1,x1k1,x3k1,uk1],[2,x1k2,x3k2,uk2],[3,x1k3,x3k3]]
#                 prediction = [[0,x1k,x3k,uk],[1,x1k1,x3k1,uk1],[2,x1k2,x3k2,uk2],[3,x1k3,x3k3,uk3],[4,x1k4,x3k4,uk4],[5,x1k5,x3k5]]  
            # Create the pandas DataFrame 
            dfOut = pd.DataFrame(prediction, columns = ['k','bpm','hp','al']) 
            filename = filetime[-1].strftime('%Y-%m-%d-%H-%M') 
            dfOut.to_csv("/Users/tomstanton 1/Documents/College/Stage 5/ME Project/Data/MPC_Predictions/"+filename+'.csv', index=False)

#         ccToEdit= pd.read_csv("/Users/tomstanton 1/Documents/College/Stage 5/ME Project/Data/CurrentCycleData.csv")
#         ccToEdit.al[currCycleReadings-1]=uk
#         ukCheck = uk+4
#         ccToEdit.to_csv("/Users/tomstanton 1/Documents/College/Stage 5/ME Project/Data/CurrentCycleData.csv", index=False)



        # OUTPUT U TO BIKE    
        print('U = ',uk)
        u.append(uk)
        t.append(filename)      
# 
# 
#
class UploadAL:
    def UploadToDropbox(u):
        dbx = dropbox.Dropbox('Dropbox key here')
        pc_path = '/Users/tomstanton 1/Documents/College/Stage 5/ME Project/Data/Bikedata/Al.csv'  
        db_path = '/AL.csv'
        dfAl = pd.DataFrame()
        dfAl['AL']  = u
        dfAl.to_csv(pc_path, index=False)
        f = open(pc_path,"rb")
        overwrite = dropbox.files.WriteMode('overwrite', None)
        dbx.files_upload(f.read(),db_path,mode=overwrite,autorename=False)
        f.close()
        
        # Rename file with new assistance level percentage (as integer) to be used by mobile app
        newpath = '/ALvalue/'+str(int(u[-1]*100))+'.csv'
        oldname = dbx.files_list_folder(path="/ALvalue",recursive=True).entries[1].name
        
        if('/ALvalue/'+oldname != newpath):
            dbx.files_move_v2('/ALvalue/'+oldname, newpath)



class Timing:
    def currentTime(filetime):
        ft = datetime.datetime.now()#  - datetime.timedelta(minutes = 1)
        filetime.append(ft)
        
        



if __name__ == "__main__":
    schedule.clear()
    x_t1 = []
    y_hr = []
    x_t3 = []
    y_hP = []
    u = []
    t = []
    filetime = []
    ukOld = []
    ukOld.append(0)
    time.sleep(65-int(strftime("%S", gmtime())))  
    print("Started at: "+strftime("%H:%M:%S", gmtime()))
    starttime=time.time()
    
    while True:
        Timing.currentTime(filetime)
        HR_Getting.E4(x_t1, y_hr,filetime)
        EBikeData_Getting.EBike(x_t3,y_hP,filetime)
        Merging_Data.MergeTable(x_t1,x_t3,y_hr,y_hP,u,t)
        SysID_MPC.ModelAndControl(u,t,ukOld,filetime, y_hr, y_hP,x_t1,x_t3)
        UploadAL.UploadToDropbox(u)
        print("Time completed = "+strftime("%H:%M:%S", gmtime())+"\n")
        time.sleep(60.0 - ((time.time()-starttime)%60.0))


KeyboardInterrupt: 